# Pandora Workshop

This workshop introduces Pandora, a comprehensive file analysis framework that provides automated malware detection and behavioral analysis through multiple security engines and analysis workers.

## What is Pandora?

Pandora is an analysis framework to discover if a file is suspicious and conveniently show the results. It integrates multiple security analysis tools and engines to provide comprehensive file intelligence, combining static analysis, behavioral analysis, and signature-based detection to identify potential threats and provide detailed security assessments.

The goal of this tool is to provide end users, particularly those receiving documents via email, with a secure, sandboxed environment to open/preview potentially unsafe files. This approach minimizes the risk of inadvertently executing malware embedded in malicious documents (e.g., PDF, DOC, DOCX, XLSM, etc.).

> **Note:** Any executable file will be automatically flagged as malicious. This is based on the assumption that end users should never run executables from untrusted or unknown sources.


https://pandora.circl.lu/submit

## Key Features

### Pandora Capabilities:
- **Multi-Engine Analysis**: Integration with multiple antivirus engines and security tools
- **Static Analysis**: File structure analysis, metadata extraction, and signature detection
- **Threat Classification**: Automated threat categorization and risk assessment
- **Worker Architecture**: Modular analysis system with specialized security workers
- **Real-Time Processing**: Automated analysis pipeline with status tracking
- **API Integration**: RESTful API for programmatic file submission and results retrieval
- **Historical Intelligence**: Cross-reference with known malware databases and threat intelligence

## Documentation

- **Project Repository**: https://github.com/pandora-analysis/pandora
- **API Documentation**: https://pandora.circl.lu/doc/
- **Python library**: https://pypandora.readthedocs.io/en/latest/index.html

### Use Cases:
- **Malware Analysis**: Automated detection and classification of malicious files
- **Incident Response**: Rapid assessment of suspicious files during security investigations
- **Threat Intelligence**: Generate IOCs and signatures from analyzed malware samples
- **Security Operations**: Integrate file analysis into security monitoring workflows
- **Digital Forensics**: Analyze files collected during forensic investigations
- **Email Security**: Scan email attachments and suspicious file downloads

## Learning Objectives

By completing this workshop, you will learn to:
- Submit files for comprehensive automated malware analysis
- Interpret multi-engine detection results and threat classifications
- Query specific security worker outputs for detailed analysis findings
- Build automated file analysis workflows using the Pandora API
- Correlate analysis results across multiple security engines for comprehensive threat assessment

## Exercises

In [ ]:
from pypandora import PyPandora

# Configure HTTP sessions for API communication
print("Configuring API client for Pandora...")

# Pandora session configuration
pandora = PyPandora()

print("API client configured successfully!")
print("Ready to query Pandora service.")

Configuring API client for Lookyloo...
API client configured successfully!
Ready to query Lookyloo service.


### Exercise 1.0: Submit a File for Analysis

**Objective:** Learn how to submit files to Pandora for comprehensive automated malware analysis and security assessment.

**Understanding File Submission:**
This exercise demonstrates the complete workflow for submitting files to Pandora:
- **File Upload**: Submit files to the analysis framework for processing
- **Task Management**: Track analysis progress through unique task identifiers
- **Status Monitoring**: Monitor analysis completion and retrieve results
- **Seed Authentication**: Use security tokens to access analysis results

**EICAR Test File:**
The exercise uses the EICAR test file, a standard antivirus test sample:
- **Industry Standard**: Universally recognized test file for antivirus validation
- **Safe Testing**: Non-malicious file designed specifically for security testing
- **Predictable Results**: Should trigger positive detection from antivirus engines
- **Educational Value**: Demonstrates analysis workflow without real malware risks

**Analysis Workflow:**
1. **File Submission**: Upload file to Pandora analysis queue
2. **Task Creation**: Receive unique task ID and authentication seed
3. **Status Monitoring**: Poll for analysis completion status
4. **Result Retrieval**: Access detailed analysis results and findings

**API Endpoints:** 
- `https://pandora.circl.lu/submit`
- `https://pandora.circl.lu/task_status`

In [13]:
from datetime import timedelta
import time

# use `eicar.com` as a sample file for testing
with open("eicar.com", "wb") as f:
    f.write(
        b"X5O!P%@AP[4\\PZX54(P^)7CC)7}$EICAR-STANDARD-ANTIVIRUS-TEST-FILE!$H+H*"
    )

print("Sample file 'eicar.com' created for testing.")

retries = 0
try:
    task = pandora.submit_from_disk("eicar.com", seed_expire=timedelta(days=1))
    print(f"File submitted successfully to Pandora for analysis: {task['link']}")
    print(f"Seed: {task['seed']}")

    status = pandora.task_status(task['taskId'], seed=task['seed'])
    while status['status'] == 'WAITING':
        if retries >= 10:
            print("Analysis is taking too long. Exiting.")
            break

        print(f"Analysis in progress... (retry #{retries})")
        retries += 1
        time.sleep(10)
        status = pandora.task_status(task['taskId'], seed=task['seed'])

    print(f"Final analysis status: {status['status']}")

    if status['status'] == 'ALERT':
        print("[ALERT]: The file has been flagged as malicious by Pandora.")

except Exception as e:
    print(f"Error submitting file to Pandora: {e}")


Sample file 'eicar.com' created for testing.
File submitted successfully to Pandora for analysis: https://pandora.circl.lu/analysis/4b9ee513-54e7-49e2-baed-edb528749cb2/seed-NP8qog9dnKkBVSyyyVldQSlxE3TgPawO85SR2_vZNmY
Seed: NP8qog9dnKkBVSyyyVldQSlxE3TgPawO85SR2_vZNmY
Analysis in progress... (retry #0)
Final analysis status: ALERT
[ALERT]: The file has been flagged as malicious by Pandora.


### Exercise 1.1: Get Worker Results (ClamAV)

**Objective:** Retrieve detailed analysis results from specific security workers to understand individual engine findings and threat classifications.

**Understanding Worker Architecture:**
Pandora uses a modular worker system where each worker provides specialized analysis:
- **ClamAV Worker**: Open-source antivirus engine with signature-based detection
- **Specialized Analysis**: Each worker focuses on specific detection techniques
- **Detailed Results**: Workers provide granular findings and threat classifications

**ClamAV Analysis Insights:**
ClamAV worker provides:
- **Signature Detection**: Identification based on known malware signatures
- **Threat Classification**: Categorization of detected threats and malware families

**API Endpoint:** `https://pandora.circl.lu/worker_status`

In [17]:
clamav_status = pandora.worker_status(task['taskId'], seed=task['seed'], worker_name='clamav', details=True)

print("CLAMAV Worker Status:")
if 'clamav' in clamav_status:
    print(f"  Status: {clamav_status['clamav']['status']}")
    print("  Details:")
    for detail in clamav_status['clamav'].get('details', []):
        print(f"    - {detail}: {clamav_status['clamav']['details'][detail]}")


CLAMAV Worker Status:
  Status: ALERT
  Details:
    - malicious: Win.Test.EICAR_HDB-1


### Exercise 1.2: Get All Enabled Workers

**Objective:** Discover all available analysis workers in Pandora to understand the comprehensive analysis capabilities and specialized detection engines available.

**Understanding Worker Ecosystem:**
Pandora's worker architecture provides diverse analysis capabilities:
- **Antivirus Engines**: Multiple commercial and open-source antivirus solutions
- **Behavioral Analysis**: Dynamic analysis and sandbox execution workers
- **Static Analysis**: File structure, metadata, and signature analysis workers
- **Threat Intelligence**: Integration with threat intelligence databases and feeds
- **Specialized Tools**: Domain-specific analysis for different file types and threats

**Worker Categories:**
- **Detection Workers**: Signature-based and heuristic malware detection
- **Analysis Workers**: Deep file analysis and behavioral assessment
- **Intelligence Workers**: Threat intelligence correlation and enrichment
- **Utility Workers**: File parsing, metadata extraction, and format analysis

**API Endpoint:** `https://pandora.circl.lu/api/enabled_workers`

In [ ]:
workers = pandora.get_enabled_workers()
print("Enabled Workers in Pandora:")
print(f"Total workers available: {len(workers)}")
print("=" * 40)

for worker in workers:
    print(f"- {worker}")

print("=" * 40)
print(f"Analysis powered by {len(workers)} specialized security workers")

Enabled Workers in Pandora:
- preview
- clamav
- misp
- outlookmsg
- odf
- blocklists
- pdf
- observables
- virustotal
- msodde
- yarahq_full
- lookyloo
- ole
- malwarebazaar
- hashlookup
- extractor
- mwdb
- yara_signature_base
- hybridanalysis
- xmldeobfuscator
- qrcode


### Homework - Multi-File Analysis with Cross-Worker Correlation

**Objective:** Create a comprehensive automated workflow that analyzes multiple files using all available Pandora workers, correlates results across engines, and generates detailed security intelligence reports.

**Challenge Requirements:**
Build an advanced file analysis pipeline that demonstrates sophisticated threat detection and intelligence correlation capabilities. Your solution should leverage multiple workers and provide actionable security intelligence.

**Technical Tasks:**
- Submit multiple test files for analysis (minimum 3 different file types)
- Query all enabled workers and understand their capabilities
- Retrieve results from multiple workers for comprehensive analysis
- Implement correlation logic to compare detection results across engines
- Generate statistical analysis of detection rates and consensus findings
- Create detailed reports with threat assessment and recommended actions

**File Analysis Set:**
Create a diverse set of test files including:
- **EICAR Test File**: Standard antivirus test sample
- **Benign Files**: Known good files for baseline comparison
- **Suspicious Samples**: Files with potential security concerns
- **Different Formats**: Various file types (executables, documents, archives)